# НИР Лабораторная работа № 2

## Дискретное преобразование Фурье

$$ \hat{f}_{k} = \sum_{n=0}^{N-1} f_n \exp\Bigl( -\dfrac{2\pi i}{N} kn \Bigr),~~~~ k = \overline{0, N-1}$$

Построить дискретные сигналы $f_n$ и соответствующие спектральные плотности $|\hat{f}_k|$ на основе функций:

1) $f(t) = \cos\bigl(\omega_1 t\bigr) + \cos\bigl(\omega_2 t\bigr)$;

2) $f(t) = \bigl(1 - \Theta\bigl( t - t_* \bigr) \bigr) \cos\bigl(\omega_1 t\bigr) + \Theta\bigl( t - t_* \bigr)\cos\bigl(\omega_2 t\bigr)$;

3) $f(t) = \cos\bigl( \omega t \bigr)$, $\omega = \omega_1 + \bigl(\omega_2 - \omega_1 \bigr)t \bigl/T$.

Невозможность отследить появление (исчезновение) или изменение частотных состовляющих сигнала следует из суммирования по всей временной реализации.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

In [2]:
def discrete_transform(array, k_list):
    def Exp(n, k):
        return np.exp(-2 * np.pi * 1j * k * n / len(array))
    
    a = [np.sum([array[n] * Exp(n, k) for n in range(0, len(array))]) for k in k_list]
    
    return np.array(a)

In [60]:
def func_1(times, w1=0.5, w2 = 2):
    return np.cos(w1 * times) + np.cos(w2 * times)


def func_2(times, w1=0.5, w2 = 2, t_ = 2):
    
    def theta(x):
        return np.array([0 if item > 0 else 1 for item in x])

    return (1 - np.heaviside(times - t_, 1)) * np.cos(w1 * times) + np.heaviside(times - t_, 1) * np.cos(w2 * times)


def func_3(times, w1=0.5, w2=2):
    
    def omega(w1, w2, t, T=20):
        return w1 + (w2 - w1) * t / T
    
    return np.cos(omega(w1, w1, times) * times)

In [61]:
times = np.arange(0, 5, 0.1)
k_list = range(0, len(times))

f1 = func_1(times, w1=5)
f2 = func_2(times)
f3 = func_3(times)

f1k = discrete_transform(f1, k_list)
f2k = discrete_transform(f2, k_list)
f3k = discrete_transform(f3, k_list)

In [5]:
def graph(times, funcs, furies):
    fig = make_subplots(rows=len(funcs), cols=2, subplot_titles=("Дискретный сигнал", "Спектральная плотность"))
    
    for row in range(len(funcs)):
        fig.add_trace(go.Bar(x=times, y=funcs[row], 
                                 name=f"f{row + 1}"), row=row + 1, col=1)
        
        fig.update_xaxes(title_text="time", row=row + 1, col=1)
        
        fig.add_trace(go.Bar(x=times, y=np.abs(furies[row]), 
                                 name=f"f{row + 1}k"), row=row + 1, col=2)
        
        fig.update_xaxes(title_text="time", row=row + 1, col=2)
        
    fig.update_layout(height=800, width=1000)
    fig.show()

In [62]:
graph(times, [f1, f2, f3], [f1k, f2k, f3k])

## Оконное Фурье-преобразование

$$ \hat{f}\bigl( \omega, t_0 \bigr) = \int\limits_{t_0 - T}^{t_0 + T} f(t) e^{-i\omega t} dt$$

1) Получить формулу дискретного оконного Фурье-преобразования.

2) Построить ряд спектров для функции

$$f(t) = \bigl(1 - \Theta\bigl( t - t_* \bigr) \bigr) \cos\bigl(\omega_1 t\bigr) + \Theta\bigl( t - t_* \bigr)\cos\bigl(\omega_2 t\bigr)$$

   при различных $t_0$ и фиксированной ширины окна $2T$.

При применении оконного преобразования Фурье, с одной стороны, необходимо выбирать ширину окна $2T$ как можно меньшей, чтобы четко выявить момент появления или исчезновения той или иной частотной составляющей сигнала; с другой стороны, при этом падает разрешение метода в частотном диапозоне.

Чтобы получить формулу дискретного оконного Фурье-преобразования необходимо сделать ряд преобразований:

- $f(t) \rightarrow f_n$;


- $\exp\bigl(-i\omega t \bigr) \rightarrow \exp\bigl(-i\omega n \bigr)$;


- $\int\limits_{t_0 - T}^{t_0 + T} \rightarrow \sum\limits_{n=0}^{N-1} w\bigl(n - m \bigr), \text{ где } w(x) =
    \begin{cases}
     0 ,|x| > T,\\ 
     1 ,|x| \leq T
    \end{cases}$ 

Формула для дискретного оконного Фурье-преобразования

$$ \hat{f}\bigl( \omega, m \bigr) = \sum_{n=0}^{N-1} f_n \exp\bigl(-i\omega n \bigr) w\bigl(n - m \bigr),~~~~ m = \overline{0, N-1}$$

In [7]:
def discrete_windowed_transform(func, m, w=2):
    
    def window(n, m, T=0.2*len(func)):
        return 0 if np.abs(n - m) > T else 1
    
    return np.sum([func[n] * np.exp(-1j * w * n) * window(n, m) for n in range(0, len(func))])

In [8]:
m_list = range(10, len(f2) - 10)
f2m = []

for m in m_list:
    f2m.append(discrete_windowed_transform(f2, m))

fig = px.bar(x=m_list, y=np.abs(f2m))
fig.show()

## Непрерывное вейвлет-преобразование

$$ W\bigl( s, t_0 \bigr) = \int\limits_{-\infty}^{+\infty} f(t) \psi_{s, t_0}^{*}(t) dt,~~~~ \psi_{s, t_0}(t) = \dfrac{1}{\sqrt{s}}\psi_0 \Bigl(  \dfrac{t - t_0}{s} \Bigr),$$

где $\psi_{s, t_0}(t)$ $-$ вейвлетная функция, $\psi_0 (t)$ $-$ материнский вейвлет, $s$ $-$ масштаб вейвлетного преобразования, $t_0$ $-$ параметр сдвига.

Построить скалограммы для дискретизированных сигналов на основе прошлых функций и сигнала динамической системы из [лабораторной работы №1](https://github.com/AlexeyMakurin/Computer-technologies-in-physics/blob/main/RW_laboratory_work_1.ipynb).

In [10]:
def wavelet(basic_wavelet, s, t0, t):
    return basic_wavelet((t - t0) / s) / np.sqrt(s)

### Примеры материнских вейвлетов

#### WAVE-вейвлет

$$\psi \bigl(t\bigr) = t e^{-t^{2} / 2} $$

In [11]:
def wave(t):
    return t * np.exp(-0.5 * t**2)

#### MHAT-вейвлет

$$\psi \bigl(t\bigr) = \bigl( 1 - t^2 \bigr) e^{-t^{2} / 2 }$$

In [13]:
def mhat(t):
    return (1 - t**2) * np.exp(-0.5 * t**2)

#### Вейвлет Морле

$$\psi \bigl(t\bigr) = \pi^{-1/4}\Bigl( e^{i2\pi f_{0} t} - e^{-(i2\pi f_{0})^2 / 2} \Bigr) e^{-t^{2}/2}, $$


где $f_0$ $-$ параметр, называемый центральной частотой. 

In [14]:
def morle(t, f0 = 1):
    a = 1j * 2 * np.pi * f0
    return (np.exp(a * t) - np.exp(-0.5 * (a)**2)) * np.exp(-0.5 * t**2) / np.pi**(0.25)

### Пример применения

 Рассмотрим гармоническую функцию ($\sin$) на основе вейвлета WAVE. 

In [15]:
time = np.arange(0, 10, 0.025)
fig = px.line(x=time, y=np.sin(5 * time))
fig.add_trace(go.Scatter(x=time, y=wavelet(wave, 0.37, 1.25, time)))

fig.show()

In [41]:
def wavelet_transform(signal, basic_wavelet, t0_max=5, s_max=2):
    
    time = np.arange(-1000, 1000, 2e-3)
    
    s_list = np.arange(1e-10, s_max, 0.05)
    t0_list = np.arange(0, t0_max, 0.05)
    
    result = np.array([
        np.trapz(y=[ signal(time) * wavelet(basic_wavelet, s , t0, time) for s in s_list], x=time, axis=1) 
        for t0 in t0_list]) 
        
    
    return result.T, s_list, t0_list

In [65]:
import time
start_time = time.time()
result, s, t0 = wavelet_transform(func_1, wave, t0_max=5, s_max=4.2)
total_time = int((time.time() - start_time))
print(total_time)

fig = px.imshow(result, x=t0, y=s, origin='lower')
fig.show()

865


In [42]:
def sin(x):
    return np.sin(5 * x)

res, s_, t0_ = wavelet_transform(sin, wave, t0_max=2, s_max=0.9)

fig = px.imshow(res, x=t0_, y=s_, origin='lower')
fig.show()

In [66]:
result, s, t0 = wavelet_transform(func_3, wave, t0_max=5, s_max=2)
fig = px.imshow(result, x=t0, y=s, origin='lower')
fig.show()

In [67]:
result, s, t0 = wavelet_transform(func_2, wave, t0_max=5, s_max=2)
fig = px.imshow(result, x=t0, y=s, origin='lower')
fig.show()